<a href="https://colab.research.google.com/github/dpredie/SingleHDR/blob/master/SingleHDR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Run Prep to clone git and dependencies


In [ ]:
!git clone https://github.com/dpredie/SingleHDR

%tensorflow_version 1.11.1
import tensorflow
print(tensorflow.__version__)

!pip install opencv-python

In [ ]:
#prep section 2 - download Large model 400 MB! - skip if running small model 

%cd /content/SingleHDR/
!gdown https://drive.google.com/uc?id=1LXP7HFoVKIC8mBPHN1VW9JnQlQPtEfbO
!unzip ckpt.zip

2. Before running please upload your 360 image to /content/input/ folder. change the below input.png name to your file name




In [3]:
# run
%cd /content/SingleHDR/
!mkdir -p output_hdrs

# Large Model - comment # to run small model
!python test_real.py --ckpt_path_deq ckpt_deq/model.ckpt --ckpt_path_lin ckpt_lin/model.ckpt --ckpt_path_hal ckpt_hal/model.ckpt --test_imgs ./imgs --output_path output_hdrs

# Small Model - uncomment to run
#!python test_real_refinement.py --ckpt_path ckpt_deq_lin_hal_ref/model.ckpt --test_imgs ./imgs --output_path output_hdrs

/content/SingleHDR
INFO:numexpr.utils:NumExpr defaulting to 2 threads.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Traceback (most recent call last):
  File "test_real.py", line 9, in <module>
    import hallucination_net
  File "/content/SingleHDR/hallucination_net.py", line 40, in <module>
    import tensorlayer as tl
ModuleNotFoundError: No module named 'tensorlayer'
